In [1]:
import mand.core

from mand.core import Entity, node, Context
from mand.core import PrintMonitor

In [2]:
class O(Entity):

    @node
    def X(self):
        return 10
    
    @node
    def Y(self):
        return self.X()**3
    
    @node
    def eps(self):
        return 1e-4
    
    @node 
    def delta(self):
        eps = self.eps()
        y = self.Y()
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump}, name='X-up')
        with c:
            y_bump = self.Y()
        return (y_bump-y)/eps
    
    @node 
    def gamma(self):
        eps = self.eps()
        d = self.delta()
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump})
        with c:
            d_bump = self.delta()
        return (d_bump-d)/eps

In [3]:
with PrintMonitor():
    o = O()
    d = o.delta()
    print 'delta:', d

 GetValue begin </Global/anon:O/05.10001>.delta()
   GetValue/Calc begin O:delta
     GetValue begin </Global/anon:O/05.10001>.eps()
       GetValue/Calc begin O:eps
     GetValue begin </Global/anon:O/05.10001>.Y()
       GetValue/Calc begin O:Y
         GetValue begin </Global/anon:O/05.10001>.X()
           GetValue/Calc begin O:X
     GetValue begin </Global/anon:O/05.10001>.X()
       GetValue from ctx </Global/anon:O/05.10001>.X()
     GetValue begin </Global/anon:O/05.10001>.eps()
       GetValue from ctx </Global/anon:O/05.10001>.eps()
     Context enter X-up
       GetValue begin </Global/anon:O/05.10001>.Y()
         GetValue/Calc begin O:Y
           GetValue begin </Global/anon:O/05.10001>.X()
             GetValue from ctx </Global/anon:O/05.10001>.X()
delta: 300.003000009


In [4]:
with PrintMonitor():
    d = o.delta()
    print 'delta:', d
    g = o.gamma()
    print 'gamma:', g

 GetValue begin </Global/anon:O/05.10001>.delta()
   GetValue from ctx </Global/anon:O/05.10001>.delta()
delta: 300.003000009
 GetValue begin </Global/anon:O/05.10001>.gamma()
   GetValue/Calc begin O:gamma
     GetValue begin </Global/anon:O/05.10001>.eps()
       GetValue from ctx </Global/anon:O/05.10001>.eps()
     GetValue begin </Global/anon:O/05.10001>.delta()
       GetValue from ctx </Global/anon:O/05.10001>.delta()
     GetValue begin </Global/anon:O/05.10001>.X()
       GetValue from ctx </Global/anon:O/05.10001>.X()
     GetValue begin </Global/anon:O/05.10001>.eps()
       GetValue from ctx </Global/anon:O/05.10001>.eps()
     Context enter 4422500248
       GetValue begin </Global/anon:O/05.10001>.delta()
         GetValue/Calc begin O:delta
           GetValue begin </Global/anon:O/05.10001>.eps()
             GetValue/Calc begin O:eps
           GetValue begin </Global/anon:O/05.10001>.Y()
             GetValue/Calc begin O:Y
               GetValue begin </Global/anon: